## **Programming for Big Data - Wild fires Project (with Bonus)** ##

Done by Marcel Penda and Coraline Best

This notebook tests all the functions created inside the library PrometheusLib and includes the bonus question 6

1) Function that cleans the dataset

In [2]:
#importing the libraries
import sys

sys.path.append("/Users/coraline/Desktop")

from PrometheusLib import wildfire

#Testing the cleaning function clean_data
file_path = "/Users/coraline/Desktop/PrometheusLib/liste_incendies_du_12_08_2022.csv"
wildfire_data = wildfire.clean_data(file_path)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/coraline/Desktop/PrometheusLib/liste_incendies_du_12_08_2022.csv'

2) Function that counts the number of fires

In [3]:
fire_counts= wildfire.count_fires(wildfire_data, selected_department= "06", selected_year= 2022)
fire_counts


,year,department,FireCount
0,2022,06,92


Functions that counts the sum of the total burnt area

In [4]:
sum_area= wildfire.sum_burnt_area(wildfire_data, selected_department="05", selected_year=2022)
sum_area

,year,department,TotalBurntArea
0,2022,05,98.0946


Functions that calculate some statistics

In [6]:
stat_burnt_area= wildfire.burnt_area_statistics(wildfire_data, selected_department="05", selected_year= 2022)
stat_burnt_area

,year,department,mean,median,Q1,Q3
0,2022,05,2.651205,0.15,0.03,0.65


3) Function that creates interactive graphs

In [16]:
#Evolution graph
graph_evolution= wildfire.evolution_graph(wildfire_data, "/Users/coraline/Desktop/Master2")
graph_evolution

In [17]:
#Pie chart
graph_pie= wildfire.pie_chart_burnt_area(wildfire_data, 2022, "/Users/coraline/Desktop/Master2" )

5) Command line

To run in the terminal:
python main.py --file_path "/Users/coraline/Desktop/PrometheusLib/liste_incendies_du_12_08_2022.csv" --year 2022 --department "06" --output_folder "/Users/coraline/Desktop"

6) Bonus


In [23]:
#Statistical test to know if the department is relevant to impact the burn area
import statsmodels.api as sm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

def department_impact(data_wildfire, year, department_code, alpha=0.05):
    """
    Compute the p-value of a statsmodels linear regression using a specific category element as a regressor
    and determine if the department has significantly higher or lower burnt areas.

    Parameters:
    - X: DataFrame containing predictor variables (including the category).
    - y: Series containing the target variable.
    - category_element: The specific category element to use as a regressor.
    - alpha: The significance level for the test (default is 0.05).

    Returns:
    - p_value: The p-value of the regression.
    - significance: A string indicating significance or not.
    """
    
    # Filter indicated year
    fire_df_year = data_wildfire[data_wildfire["year"] == year]

    # Create np array for burnt area
    y = fire_df_year["surface_ha"].values

    # Create subset with department catagories
    X_df = fire_df_year[["department"]]

    # Create dummy variables for the department variable using One-hot-encode
    X_dummy = pd.get_dummies(X_df, columns=["department"])

    # Keep only column of specified department 
    X_dummy = X_dummy[[col for col in X_dummy.columns if f'department_{department_code}' == col]]

    # Create np-array for department values
    X = X_dummy.values

    # Add a constant term to the predictor matrix (intercept term)
    X = sm.add_constant(X)

    # Create a linear regression model
    model = sm.OLS(y, X)

    # Fit the model to the data
    results = model.fit()

    summary = results.summary()

    # Define all possible categories
    #categories = data["department"].unique()

    # Get index of department of interest
    #department_index = next((i for i, x in enumerate(categories) if x == department_code), None)

    # Get the p-value for the category element
    p_value = results.pvalues[1]

    # Get the coefficient for the category element indicating how much more area was burnt in this department compared to the average of all the others. 
    coeff = results.params[1]

    # Determine significance based on the alpha threshold
    if p_value <= alpha and coeff > 0:
        significance = f"In {year} With a p-value of {round(p_value,3)}, the department {department_code} observed {round(coeff, 3)} (coefficient) times more burnt areas compared to the average of all other departments."
    
    elif p_value <= alpha and coeff < 0:
        significance = f"In {year} With a p-value of {round(p_value,3)}, the department {department_code} observed {round(coeff, 3)} times less burnt areas compared to the average of all other departments."
    else:
        significance = f"In {year}, with a p-value of {round(p_value,3)}, the department {department_code} did not have significantly different burnt areas compared to the average of all other departments."

    return significance

In [24]:
# Check if function works
department_impact(wildfire_data, 2022, "04", alpha=0.05)

'In 2022, with a p-value of 0.21, the department 04 did not have significantly different burnt areas compared to the average of all other departments.'